In [19]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Data Preparation
data = {
    'Price_B': [70, 61, 55, 65, 59, 64, 68, 69, 68],
    'Price_S': [75, 79, 77, 76, 72, 70, 81, 74, 81],
    'Price_P': [90, 100, 98, 96, 95, 92, 90, 101, 90],
    'Choice':  ['B', 'P', 'N', 'S', 'B', 'P', 'P', 'S', 'P']
}
df = pd.DataFrame(data)

df['B'] = (df['Choice'] == 'B').astype(int)
df['S'] = (df['Choice'] == 'S').astype(int)
df['P'] = (df['Choice'] == 'P').astype(int)

X = df[['Price_B', 'Price_S', 'Price_P']].values
Y = df[['B', 'S', 'P']].values

# 1. MLE for WTP and alpha
def likelihood_fun(params, X, Y):
    WTP_s = params[0:3]
    alpha = params[3]
    
    # Utility: V = WTP - alpha * Price
    V_B = WTP_s[0] - alpha * X[:, 0]
    V_S = WTP_s[1] - alpha * X[:, 1]
    V_P = WTP_s[2] - alpha * X[:, 2]
    
    # Log-likelihood calculation
    log_sum_exp = np.log(1 + np.exp(V_B) + np.exp(V_S) + np.exp(V_P))
    log_likelihood = np.sum(V_B * Y[:, 0] + V_S * Y[:, 1] + V_P * Y[:, 2]) - np.sum(log_sum_exp)
    
    return -log_likelihood

# Optimization settings
initial_guess = np.array([7.0, 8.0, 10.0, 0.1])
bounds = [(0.1, 100), (0.1, 100), (0.1, 100), (0.01, 2.0)]

result = minimize(likelihood_fun, x0=initial_guess, args=(X, Y), method='L-BFGS-B', bounds=bounds)

# Parameters extraction
WTP_scaled = result.x[0:3]
alpha_est = result.x[3]
WTP_real = WTP_scaled / alpha_est

# ================================
# Question (2): Choice Probabilities
# ================================

# Since customers are assumed to be homogeneous,
# we compute choice probabilities using ONE representative price vector.
# A standard choice is the sample mean prices.

mean_prices = X.mean(axis=0)  # [mean_Price_B, mean_Price_S, mean_Price_P]

# Utilities evaluated at the representative prices
V_B_rep = WTP_scaled[0] - alpha_est * mean_prices[0]
V_S_rep = WTP_scaled[1] - alpha_est * mean_prices[1]
V_P_rep = WTP_scaled[2] - alpha_est * mean_prices[2]

# Multinomial Logit choice probabilities
denominator = 1 + np.exp(V_B_rep) + np.exp(V_S_rep) + np.exp(V_P_rep)

prob_B = np.exp(V_B_rep) / denominator
prob_S = np.exp(V_S_rep) / denominator
prob_P = np.exp(V_P_rep) / denominator
prob_N = 1 / denominator  # No-purchase option (utility normalized to 0)

# Final Outputs
print('########################################################')
print('Question (1): Parameter Estimates')
print(f'Alpha (Common): {alpha_est:.4f}')
print(f'WTP Basic:      {WTP_real[0]:.2f}')
print(f'WTP Standard:   {WTP_real[1]:.2f}')
print(f'WTP Premium:    {WTP_real[2]:.2f}')
print('Question (2): Choice Probabilities')
print(f'Prob(Basic):    {prob_B:.4f}')
print(f'Prob(Standard): {prob_S:.4f}')
print(f'Prob(Premium):  {prob_P:.4f}')
print(f'Prob(None):     {prob_N:.4f}')
print('########################################################')

########################################################
Question (1): Parameter Estimates
Alpha (Common): 0.0751
WTP Basic:      72.81
WTP Standard:   85.01
WTP Premium:    112.89
Question (2): Choice Probabilities
Prob(Basic):    0.2156
Prob(Standard): 0.2225
Prob(Premium):  0.4478
Prob(None):     0.1141
########################################################
